## Fine-tuning vs Transfer-learning vs backbone
* 딥러닝을 공부하다보면 backbone, fine-tuning, transfer-learning에 대한 용어가 자주 등장합니다
* 이 용어들의 차이점을 알아봅시다

https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/

## Fine-tuning
* 모델의 layer가 많아질수록 학습되는 parameter의 양이 많아지고 학습시간이 증가하게 됩니다
* 성능이 좋|은 대부분의 모델엔 수 많은 layer가 있고 학습시간이 수십일까지 걸리기도 합니다
* 공개된 모델과 동일한 용도로 사용할 때는 그냥 가져다쓰면 되지만 다른 용도로 사용하고 싶으면 어떡해야 할까요?
* 예들을면 30가지 클래스를 분류하는 Object detection model을 개, 고양이 분류 모델로 만들고 싶으면요?
* 직접 custom dataset을 구축하고 처음부터 모델을 학습해서 수십일의 시간을 써야할까요?
* 이 때 사용하는 것이 `fine-tuning`입니다
* `VGGnet`과 같이 이미 학습된 모델에서 feature map 생성부분인 CNN layer와, 분류 부분인 FC layer (Fully-connected layer)만 용도에 맞게 수정해주는 것입니다
* 이 때 classification이 아니라 object detection이나 segmentation을 수행하는 모델을 만든다면, CNN layer만 가져다 사용할 수 있는데, 이 CNN layer를 backbone이라 합니다

### 1. Large, Different Dataset
* pre-trained model 학습에 사용된 dataset과 많이 다른 dataset이 있을 때 사용하는 방법입니다
* 기존 학습에 사용된 learning-rate의 1/10으로 lr을 적용하여 backbone과 FC layer를 학습합니다
* lr을 너무 크게 조정하면 모델이 완전이 새로 학습될 수 있습니다

### 2. Large, Similar Dataset
* Conv. layer는 input과 가까울수록 선, 색 등의 일반적인 특징을 학습하고
* output과 가까워질수록 class 분류와 관련된 세세한 특징들을 학습합니다
* 유사한 dataset을 사용한다면 시간절약을 위해 Conv. layer의 후반 계층과 FC layer만 학습합니다
    * 학습하지 않는 부분은 lr=0, 학습하는 부분은 lr = 기존의 10%로 지정합니다

### 3. Small, Different Dataset
* 가장 challenge한 경우입니다
* 데이터가 적기 때문에 overfitting의 위험성이 있어 Conv. layer 전부 학습하는 건 피해야 합니다
* Conv. layer에서 어느정도 계층만 학습할 지 `잘` 정해서 학습해야 합니다
* classification을 담당하는 FC layer는 모두 학습합니다 (data augmentation을 이용할 수 있음)

### 4. Small, Similar Dataset
* Conv. layer는 학습하지 않고 (Overfitting), FC layer만 학습합니다

## 주의할 점
* Fine-tuning은 이미 학습된 안정적인 가중치를 `조금씩` 바꾸는 방법입니다
* 따라서 새로운 layer를 붙이는 것에 주의를 기울여야 합니다
    * learning rate를 작게 설정하기 (기존의 1/10)
    * 안정성있는 optimizer 사용하기 (SGD 등)
    * bottleneck feature만 따로 저장해서 FC layer를 학습시키기 (데이터가 충분히 있을 때만)
    * 모든 layer는 1번이상 학습이 완료된 상태여야 함 (random weight가 할당되어서는 안됨)

## Transfer Learning
* 거의 transfer learning = fine-tuning이라 봐도 무방
* 가령 object detection이나 segmentation 문제에서 resnet CNN에서 classification (FC layer)를 제거하고
* feature extraction으로 사용하면 resnet의 CNN을 backbone으로 썼다라고 함